MDA_HW2_Report
106062132 趙仰生
import library

In [12]:
from pyspark import SparkConf , SparkContext
import heapq

第一個mapper，key為出發node，value則是被指到的node

In [13]:
def mapper1(line):
    wordlist = line.split("\t")
    maplist = []
    key = wordlist[0]
    value = [wordlist[1]]
    maplist.append((key,value))
    return maplist

用助教教的方法去平行化，跑得更快。
然後將資料經過第一個mapper處理，並且把r[]建出來（裡頭的初始值是1/10876）。

In [14]:
conf=SparkConf().set("spark.default.parallelism", 4)
sc = SparkContext(conf=conf)
lines = sc.textFile("p2p-Gnutella04.txt").flatMap(mapper1)
r = [1/10876] * 10876
breakcheck = r

這個reducer從頭用到尾，直接把value值相加

In [15]:
def reducer1(x,y):
    return x+y

把剛剛map完的lines做reduce

In [16]:
lines = lines.reduceByKey(reducer1)

第二個mapper將contrbution算出來，key為被指向的node，value[0]為出發node，value[1]為contribution再乘以Beta(0.8)

In [17]:
def mapper2(line):
    maplist = []
    ct1 = 0
    for item in line[1]:
        ct1 = ct1 + 1
    ct1 = 1 / ct1
    for item2 in line[1]:
        key = item2
        value = [line[0],ct1*0.8]
        maplist.append((key,value))
    return maplist

將lines經過第二個mapper處理，並且reduce。
並且把這個lines存給lines2（方便後續iteration處理）

In [18]:
lines = lines.flatMap(mapper2)
lines = lines.reduceByKey(reducer1)
lines2 = lines

第三個mapper將contribution乘上rank值，所以key會是被指向的node，value會是剛剛的contribution乘上出發node的值。
這裡有做一個特殊處理，因為測資有三個空的點(10452,10493,10647)，而且有三個超出10876的點(10876,10877,10878)，所以將後面的三個填到前面的空缺。

In [19]:
def mapper3(line):
    maplist = []
    for i in range(0,len(line[1]),2):
        key = int(line[0])
        if(int(line[1][i]) == 10876):
            value = line[1][i+1]*r[10452]
        if(int(line[1][i]) == 10877):
            value = line[1][i+1]*r[10493]
        if(int(line[1][i]) == 10878):
            value = line[1][i+1]*r[10647]
        else:
            value = line[1][i+1]*r[int(line[1][i])]
        maplist.append((key,value))
    return maplist

這裡是iteration，首先把剛剛存好的lines2丟回給lines，這樣就可以跑一樣的mapper3（其中不一樣的只有不斷被更新的rank值）
然後最後做reduce把所有的值加總，將所有的點的rank設成0（避免初始值影響）
並且用take把加總的值拿出來存回去r[]裡面（這裡一樣把後面超出範圍的三個點填到前面空著的三個點），然後將1/N*(1-B)加到每個點上
最後用sum做renormalize，並且iteration20次就完成了。

In [9]:
for n in range(20):
    lines = lines2
    lines = lines.flatMap(mapper3)
    lines = lines.reduceByKey(reducer1)
    lines_list = lines.take(10876)
    rsum = 0
    for i in range(10876):
        r[i] = 0
    for item in lines_list:
        if(item[0] == 10876):
            r[10452] = item[1]
        elif(item[0] == 10877):
            r[10493] = item[1]
        elif(item[0] == 10878):
            r[10647] = item[1]
        else:
            r[item[0]] = item[1]
    for i in range(10876):
        r[i] += 1/10876*0.2
        rsum += r[i]
    renorma = (1 - rsum)/10876
    for i in range(10876):
        r[i] += renorma

把前10的值印出來。

In [10]:
res = sorted(range(len(r)), key = lambda sub: r[sub])[-10:] 
for i in range(10):
    print(res[9-i],r[res[9-i]])

1056 0.0006321988095901951
1054 0.0006291557128603996
1536 0.000523910339752709
171 0.0005116224706020393
453 0.0004956586476699704
407 0.0004848441996390273
263 0.00047961928931848473
4664 0.0004704975514074384
261 0.00046289158656890255
410 0.0004615100382904277


寫入Outputfile，並且stop sc。

In [11]:
f = open('Outputfile.txt', 'w')
for i in range(10):
    f.write('{:6}'.format(str(res[9-i]))+str(r[res[9-i]])+'\n')
f.close()
sc.stop()